In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

pd.set_option('display.max_columns', None)

In [3]:
#df = pd.read_csv('./bank-loan-status-dataset/credit_train.csv')

In [1]:
# Columns from kaggle dataset

In [4]:
df.columns

Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')

# Drop unnecessary columns:

In [67]:
#df = pd.read_csv('~/Downloads/LoanStats3c.csv',header = 1)
#df.to_pickle('./data/loan_data_2014.zip',compression = 'zip')

In [16]:
df = pd.read_pickle('./data/loan_data_2014.zip',compression = 'zip')

In [17]:
# add categorical variable - HARdship END FLAG EXISTS
# THIS EXISTS AS HARDSHIP_STATUS

In [18]:
df=df.drop(columns = [
        # Only looking at loans for individuals, not joint
        'annual_inc_joint',
        'application_type',
        'dti_joint',
        'verification_status_joint',
        'revol_bal_joint', 


        # Don't need specific ID info
        'id',
        'member_id',
    
        # Only care about information available pre loan
        'collection_recovery_fee',
        'collections_12_mths_ex_med',
        # Hard to classify
        'emp_title',
    
        # Keeping the hardship flag but looking at pre-loan metrics
        'hardship_type',
        'hardship_reason',
        'deferral_term',
        'hardship_amount',
        'hardship_start_date',
        'hardship_end_date',
        'payment_plan_start_date',
        'hardship_length',
        'hardship_dpd',
        'hardship_loan_status',
        'orig_projected_additional_accrued_interest',
        # Trying to extrapolate beyond certain issuing dates of loans
        'issue_d',
        'mths_since_last_major_derog',
        'mths_since_last_record',
        'num_tl_30dpd',
        'num_tl_120dpd_2m',
        
        'debt_settlement_flag',
        'debt_settlement_flag_date',
        'desc',
    
        # This is for investors / loan funding
        'initial_list_status',
        'funded_amnt',
        'funded_amnt_inv',              
        # Probably useful metrics, but all NaN in this dataset              
        'open_acc_6m',                 
        'open_il_12m',
        'open_il_24m',                     
        'open_act_il',   
        'open_rv_12m',
        'open_rv_24m',
                      
        'out_prncp', 
        'out_prncp_inv',
        'policy_code',
        'recoveries',
        # Columns related to debt settlement; we only care about pre loan features              
        'settlement_status',
        'settlement_date',
        'settlement_amount',
        'settlement_percentage',
        'settlement_term',
        'title',
        'total_cu_tl',
                      
        'total_pymnt',
        'total_pymnt_inv',
        'url',
    
        'zip_code'])



In [19]:
df.shape

(235631, 92)

In [21]:
df.sample(5)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,acc_now_delinq,tot_coll_amt,tot_cur_bal,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_status,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method
148447,14400.0,36 months,10.99%,471.37,B,B3,3 years,MORTGAGE,48000.0,Not Verified,Fully Paid,n,debt_consolidation,CO,13.88,0.0,Dec-1989,1.0,43.0,14.0,1.0,12564.0,73%,32.0,14400.0,2564.45,0.0,Apr-2017,1409.24,NaN,Mar-2018,0.0,0.0,184985.0,NaN,NaN,NaN,NaN,NaN,17200.0,NaN,NaN,6.0,15415.0,1305.0,84.8,0.0,0.0,167.0,293.0,2.0,2.0,6.0,2.0,NaN,3.0,NaN,1.0,4.0,9.0,4.0,6.0,8.0,11.0,18.0,9.0,14.0,0.0,3.0,90.6,75.0,1.0,0.0,209120.0,26064.0,8600.0,16520.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash
176359,14400.0,36 months,11.99%,478.22,B,B3,8 years,MORTGAGE,300000.0,Source Verified,Fully Paid,n,home_improvement,IL,11.09,3.0,May-1996,2.0,16.0,13.0,0.0,22848.0,57.4%,41.0,14400.0,2819.67,0.0,May-2017,478.10,NaN,Mar-2019,0.0,163.0,390727.0,NaN,NaN,NaN,NaN,NaN,39800.0,NaN,NaN,4.0,32561.0,10152.0,69.2,0.0,0.0,215.0,164.0,3.0,3.0,5.0,3.0,70.0,1.0,22.0,1.0,2.0,2.0,3.0,11.0,16.0,8.0,20.0,2.0,13.0,1.0,3.0,82.9,66.7,0.0,0.0,457606.0,179083.0,33000.0,204138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash
223341,20000.0,36 months,14.98%,693.12,C,C3,10+ years,MORTGAGE,85000.0,Source Verified,Fully Paid,n,debt_consolidation,NY,6.58,0.0,Jun-2001,3.0,63.0,11.0,0.0,10518.0,54.5%,24.0,20000.0,4758.82,0.0,Jul-2016,5351.46,NaN,Mar-2019,0.0,0.0,20317.0,NaN,NaN,NaN,NaN,NaN,19300.0,NaN,NaN,1.0,1847.0,3487.0,66.1,0.0,0.0,151.0,88.0,11.0,11.0,0.0,11.0,NaN,1.0,NaN,1.0,5.0,7.0,5.0,7.0,8.0,9.0,14.0,7.0,11.0,0.0,1.0,87.5,40.0,0.0,0.0,33806.0,20317.0,10300.0,12882.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash
21134,8750.0,36 months,10.49%,284.36,B,B3,1 year,OWN,26000.0,Source Verified,Fully Paid,n,home_improvement,MI,21.77,0.0,Aug-2001,0.0,38.0,7.0,0.0,7696.0,42.5%,18.0,8750.0,1499.59,0.0,Dec-2017,284.90,NaN,Mar-2019,0.0,0.0,15392.0,NaN,NaN,NaN,NaN,NaN,18100.0,NaN,NaN,8.0,2199.0,998.0,47.5,0.0,0.0,159.0,137.0,5.0,5.0,0.0,20.0,38.0,7.0,38.0,0.0,1.0,4.0,1.0,4.0,6.0,6.0,12.0,4.0,7.0,0.0,3.0,94.4,0.0,0.0,0.0,28100.0,15392.0,1900.0,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Cash
183809,30000.0,60 months,18.25%,765.89,D,D3,10+ years,RENT,68750.0,Verified,Fully Paid,n,credit_card,IN,33.85,0.0,Mar-1998,0.0,NaN,18.0,0.0,29787.0,59.2%,27.0,30000.0,11694.37,0.0,Nov-2016,18717.67,NaN,Nov-2016,0.0,821.0,51873.0,NaN,NaN,NaN,NaN,NaN,50300.0,NaN,NaN,2.0,2882.0,9996.0,74.3,0.0,0.0,68.0,192.0,17.0,17.0,0.0,17.0,NaN,9.0,NaN,0.0,8.0,10.0,8.0,12.0,7.0,14.0,20.0,10.0,18.0,0.0,0.0,100.0,62.5,0.0,0.0,110394.0,51873.0,38900.0,60094.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,Ca

In [123]:
#df[df['num_tl_120dpd_2m'] >= 1][['num_tl_30dpd','num_tl_120dpd_2m','num_tl_90g_dpd_24m']]

In [47]:
# df[['int_rate','grade','sub_grade']].sort_values(by = 'int_rate')

In [33]:
# Clean columns
df['term'] = df['term'].str.replace('36 months','short').str.replace('60 months','long').str.strip()
df['int_rate'] = df['int_rate'].astype(str).str[:-1]

In [37]:
df.dtypes

loan_amnt                              float64
term                                    object
int_rate                                object
installment                            float64
grade                                   object
sub_grade                               object
emp_length                              object
home_ownership                          object
annual_inc                             float64
verification_status                     object
loan_status                             object
pymnt_plan                              object
purpose                                 object
addr_state                              object
dti                                    float64
delinq_2yrs                            float64
earliest_cr_line                        object
inq_last_6mths                         float64
mths_since_last_delinq                 float64
open_acc                               float64
pub_rec                                float64
revol_bal    